# Hybrid Balanced Configuration Analysis with Per-Bin Accuracy

This notebook analyzes the hybrid balanced configuration that combines anti-overfitting features for magnitude with high-performance features for frequency.

## Overview
- Loads hybrid balanced trained models
- Analyzes anti-overfitting effectiveness for magnitude
- Evaluates high-performance features for frequency
- Provides per-bin accuracy analysis
- Compares Simple LSTM vs Attention LSTM performance

In [ ]:
import sys
import os
import json
import subprocess
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple

# Add src to path
sys.path.append('src')

# Set style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## Load Hybrid Balanced Configuration

In [ ]:
# Load the hybrid balanced configuration
with open('hybrid_balanced_config.json', 'r') as f:
    hybrid_config = json.load(f)

print("🔬 HYBRID BALANCED CONFIGURATION")
print("=" * 60)
print(f"Name: {hybrid_config['name']}")
print(f"Description: {hybrid_config['description']}")
print(f"Version: {hybrid_config['version']}")
print()

print("🏗️  MODEL ARCHITECTURE:")
for key, value in hybrid_config['model_architecture'].items():
    print(f"  {key}: {value}")
print()

print("⚙️  TRAINING PARAMETERS:")
for key, value in hybrid_config['training_parameters'].items():
    print(f"  {key}: {value}")
print()

print("�� LOSS WEIGHTS:")
for key, value in hybrid_config['loss_weights'].items():
    print(f"  {key}: {value}")
print()

print("📊 FREQUENCY SCALING:")
for key, value in hybrid_config['frequency_scaling'].items():
    print(f"  {key}: {value}")

## Analyze Hybrid Features

In [ ]:
print("🔬 HYBRID FEATURES ANALYSIS")
print("=" * 50)

print("🎯 Magnitude (Anti-Overfitting Features):")
for feature, enabled in hybrid_config['hybrid_features']['magnitude_anti_overfitting'].items():
    status = "✅" if enabled else "❌"
    print(f"  {status} {feature}")

print("\n🚀 Frequency (High-Performance Features):")
for feature, enabled in hybrid_config['hybrid_features']['frequency_high_performance'].items():
    status = "✅" if enabled else "❌"
    print(f"  {status} {feature}")

print("\n⚖️ Balanced Training Features:")
for feature, enabled in hybrid_config['hybrid_features']['balanced_training'].items():
    status = "✅" if enabled else "❌"
    print(f"  {status} {feature}")

print(f"\n📊 Expected Performance:")
print(f"  Magnitude Accuracy: {hybrid_config['expected_behavior']['magnitude_accuracy_target']}")
print(f"  Frequency Accuracy: {hybrid_config['expected_behavior']['frequency_accuracy_target']}")
print(f"  Prediction Range: {hybrid_config['expected_behavior']['prediction_range_target']}")
print(f"  Overfitting Indicators: {hybrid_config['expected_behavior']['overfitting_indicators']}")
print(f"  Performance Balance: {hybrid_config['expected_behavior']['performance_balance']}")

## Set Output Directory (Use Hybrid Balanced Results)

In [ ]:
# Use hybrid balanced results
output_dir = "data/results_hybrid_balanced_comparison"

print(f"Output directory: {output_dir}")
print(f"Full path: {Path(output_dir).absolute()}")
print(f"Results dir: {Path(output_dir) / 'results' / 'model_comparison'}")
print(f"Results dir exists: {(Path(output_dir) / 'results' / 'model_comparison').exists()}")

## Load and Analyze Results

In [ ]:
def load_comparison_results(output_dir: str) -> Dict:
    """Load the comparison results from the output directory."""
    
    results_dir = Path(output_dir) / "results" / "model_comparison"
    
    if not results_dir.exists():
        print(f"❌ Results directory not found: {results_dir}")
        return None
    
    print(f"✅ Found results directory: {results_dir}")
    
    # List available files
    available_files = list(results_dir.glob("*"))
    print(f"�� Available files:")
    for file in available_files:
        print(f"  - {file.name}")
    
    # Load comparison metrics
    metrics_file = results_dir / "comparison_metrics.json"
    if metrics_file.exists():
        with open(metrics_file, 'r') as f:
            metrics = json.load(f)
        print(f"\n📊 Loaded comparison metrics:")
        print(f"  Simple LSTM - Magnitude Accuracy: {metrics['simple_lstm_metrics']['magnitude_accuracy']:.3f}")
        print(f"  Simple LSTM - Frequency Accuracy: {metrics['simple_lstm_metrics']['frequency_accuracy']:.3f}")
        print(f"  Attention LSTM - Magnitude Accuracy: {metrics['attention_lstm_metrics']['magnitude_accuracy']:.3f}")
        print(f"  Attention LSTM - Frequency Accuracy: {metrics['attention_lstm_metrics']['frequency_accuracy']:.3f}")
    
    # Check if trained models exist
    simple_model_path = results_dir / "simple_lstm_model.pth"
    attention_model_path = results_dir / "attention_lstm_model.pth"
    
    if simple_model_path.exists() and attention_model_path.exists():
        print(f"\n✅ Found trained models:")
        print(f"  Simple LSTM: {simple_model_path}")
        print(f"  Attention LSTM: {attention_model_path}")
        return {
            'models_dir': str(results_dir),
            'simple_model_path': str(simple_model_path),
            'attention_model_path': str(attention_model_path),
            'metrics': metrics if 'metrics' in locals() else None
        }
    else:
        print(f"\n❌ Trained models not found")
        return None

# Load results if available
if output_dir:
    results = load_comparison_results(output_dir)
else:
    results = None

## Hybrid Config Performance Analysis

In [ ]:
if results and results.get('metrics'):
    metrics = results['metrics']
    
    print("🔬 HYBRID CONFIG PERFORMANCE ANALYSIS")
    print("=" * 60)
    
    # Extract metrics
    simple_mag_acc = metrics['simple_lstm_metrics']['magnitude_accuracy']
    simple_freq_acc = metrics['simple_lstm_metrics']['frequency_accuracy']
    attention_mag_acc = metrics['attention_lstm_metrics']['magnitude_accuracy']
    attention_freq_acc = metrics['attention_lstm_metrics']['frequency_accuracy']
    
    print("📊 ACCURACY ANALYSIS:")
    print(f"  Simple LSTM:     {simple_mag_acc:.3f} magnitude, {simple_freq_acc:.3f} frequency")
    print(f"  Attention LSTM:  {attention_mag_acc:.3f} magnitude, {attention_freq_acc:.3f} frequency")
    
    # Check anti-overfitting effectiveness
    print("\n🛡️ ANTI-OVERFITTING CHECK:")
    max_freq_acc = max(simple_freq_acc, attention_freq_acc)
    if max_freq_acc < 0.95:
        print(f"  ✅ Frequency NOT overfitted: {max_freq_acc:.3f} < 0.95")
    else:
        print(f"  ⚠️ Frequency may be overfitted: {max_freq_acc:.3f} >= 0.95")
    
    # Check performance balance
    print("\n⚖️ PERFORMANCE BALANCE:")
    simple_total = simple_mag_acc + simple_freq_acc
    attention_total = attention_mag_acc + attention_freq_acc
    
    if abs(simple_total - attention_total) < 0.1:
        print("  ✅ Models are well balanced")
    else:
        print("  ⚠️ Models show performance imbalance")
    
    # Hybrid config success assessment
    print("\n🎯 HYBRID CONFIG SUCCESS:")
    success_criteria = [
        max_freq_acc < 0.95,  # No overfitting
        min(simple_mag_acc, attention_mag_acc) > 0.6,  # Good magnitude performance
        min(simple_freq_acc, attention_freq_acc) > 0.7,  # Good frequency performance
    ]
    
    success_count = sum(success_criteria)
    if success_count == 3:
        print("  🏆 HYBRID CONFIG: FULL SUCCESS!")
    elif success_count == 2:
        print("  🎯 HYBRID CONFIG: MOSTLY SUCCESSFUL")
    else:
        print("  ⚠️ HYBRID CONFIG: NEEDS ADJUSTMENT")
    
    # Detailed analysis
    print("\n📋 DETAILED ANALYSIS:")
    print(f"  Magnitude Performance: {'✅ Good' if min(simple_mag_acc, attention_mag_acc) > 0.6 else '⚠️ Needs improvement'}")
    print(f"  Frequency Performance: {'✅ Good' if min(simple_freq_acc, attention_freq_acc) > 0.7 else '⚠️ Needs improvement'}")
    print(f"  Overfitting Prevention: {'✅ Effective' if max_freq_acc < 0.95 else '⚠️ May be overfitting'}")
    print(f"  Model Balance: {'✅ Balanced' if abs(simple_total - attention_total) < 0.1 else '⚠️ Imbalanced'}")
else:
    print("⚠️  No metrics available to analyze.")

## Display Overall Comparison Metrics

In [ ]:
if results and results.get('metrics'):
    metrics = results['metrics']
    
    print("�� OVERALL COMPARISON METRICS (HYBRID CONFIG)")
    print("=" * 60)
    
    # Simple LSTM metrics
    print("\n🔵 Simple LSTM Performance:")
    print(f"  Total Loss: {metrics['simple_lstm_metrics']['total_loss']:.4f}")
    print(f"  Magnitude Loss: {metrics['simple_lstm_metrics']['magnitude_loss']:.4f}")
    print(f"  Frequency Loss: {metrics['simple_lstm_metrics']['frequency_loss']:.4f}")
    print(f"  Magnitude Accuracy: {metrics['simple_lstm_metrics']['magnitude_accuracy']:.3f}")
    print(f"  Frequency Accuracy: {metrics['simple_lstm_metrics']['frequency_accuracy']:.3f}")
    print(f"  Magnitude Correlation: {metrics['simple_lstm_metrics']['magnitude_corr']:.3f}")
    print(f"  Frequency Correlation: {metrics['simple_lstm_metrics']['frequency_corr']:.3f}")
    
    # Attention LSTM metrics
    print("\n🟡 Attention LSTM Performance:")
    print(f"  Total Loss: {metrics['attention_lstm_metrics']['total_loss']:.4f}")
    print(f"  Magnitude Loss: {metrics['attention_lstm_metrics']['magnitude_loss']:.4f}")
    print(f"  Frequency Loss: {metrics['attention_lstm_metrics']['frequency_loss']:.4f}")
    print(f"  Magnitude Accuracy: {metrics['attention_lstm_metrics']['magnitude_accuracy']:.3f}")
    print(f"  Frequency Accuracy: {metrics['attention_lstm_metrics']['frequency_accuracy']:.3f}")
    print(f"  Magnitude Correlation: {metrics['attention_lstm_metrics']['magnitude_corr']:.3f}")
    print(f"  Frequency Correlation: {metrics['attention_lstm_metrics']['frequency_corr']:.3f}")
    
    # Hyperparameters used
    print("\n⚙️  Hyperparameters Used:")
    for key, value in metrics['hyperparameters'].items():
        print(f"  {key}: {value}")
    
    # Performance comparison
    print("\n�� PERFORMANCE COMPARISON:")
    
    # Magnitude accuracy comparison
    simple_mag_acc = metrics['simple_lstm_metrics']['magnitude_accuracy']
    attention_mag_acc = metrics['attention_lstm_metrics']['magnitude_accuracy']
    
    if simple_mag_acc > attention_mag_acc:
        improvement = ((simple_mag_acc - attention_mag_acc) / attention_mag_acc) * 100
        print(f"  🎯 Simple LSTM wins on Magnitude Accuracy: {simple_mag_acc:.3f} vs {attention_mag_acc:.3f} (+{improvement:.1f}%)")
    else:
        improvement = ((attention_mag_acc - simple_mag_acc) / simple_mag_acc) * 100
        print(f"  🎯 Attention LSTM wins on Magnitude Accuracy: {attention_mag_acc:.3f} vs {simple_mag_acc:.3f} (+{improvement:.1f}%)")
    
    # Frequency accuracy comparison
    simple_freq_acc = metrics['simple_lstm_metrics']['frequency_accuracy']
    attention_freq_acc = metrics['attention_lstm_metrics']['frequency_accuracy']
    
    if simple_freq_acc > attention_freq_acc:
        improvement = ((simple_freq_acc - attention_freq_acc) / attention_freq_acc) * 100
        print(f"  🎯 Simple LSTM wins on Frequency Accuracy: {simple_freq_acc:.3f} vs {attention_freq_acc:.3f} (+{improvement:.1f}%)")
    else:
        improvement = ((attention_freq_acc - simple_freq_acc) / simple_freq_acc) * 100
        print(f"  🎯 Attention LSTM wins on Frequency Accuracy: {attention_freq_acc:.3f} vs {simple_freq_acc:.3f} (+{improvement:.1f}%)")
    
    # Overall winner
    simple_total = simple_mag_acc + simple_freq_acc
    attention_total = attention_mag_acc + attention_freq_acc
    
    if simple_total > attention_total:
        print(f"\n🏆 OVERALL WINNER: Simple LSTM ({simple_total:.3f} vs {attention_total:.3f})")
    else:
        print(f"\n🏆 OVERALL WINNER: Attention LSTM ({attention_total:.3f} vs {simple_total:.3f})")
else:
    print("⚠️  No metrics available to analyze.")

## Create Hybrid Performance Visualizations

In [ ]:
if results and results.get('metrics'):
    metrics = results['metrics']
    
    # Create comprehensive comparison plots
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(18, 14))
    
    # 1. Accuracy Comparison
    models = ['Simple LSTM', 'Attention LSTM']
    mag_acc = [metrics['simple_lstm_metrics']['magnitude_accuracy'], 
               metrics['attention_lstm_metrics']['magnitude_accuracy']]
    freq_acc = [metrics['simple_lstm_metrics']['frequency_accuracy'], 
                metrics['attention_lstm_metrics']['frequency_accuracy']]
    
    x = np.arange(len(models))
    width = 0.35
    
    bars1 = ax1.bar(x - width/2, mag_acc, width, label='Magnitude Accuracy', alpha=0.8, color='skyblue')
    bars2 = ax1.bar(x + width/2, freq_acc, width, label='Frequency Accuracy', alpha=0.8, color='lightcoral')
    ax1.set_xlabel('Model')
    ax1.set_ylabel('Accuracy')
    ax1.set_title('Hybrid Config: Accuracy Comparison', fontweight='bold')
    ax1.set_xticks(x)
    ax1.set_xticklabels(models)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, 1)
    
    # Add value labels on bars
    for bar in bars1:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.02,
                f'{height:.3f}', ha='center', va='bottom', fontweight='bold')
    
    for bar in bars2:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.02,
                f'{height:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # 2. Loss Comparison
    total_loss = [metrics['simple_lstm_metrics']['total_loss'], 
                  metrics['attention_lstm_metrics']['total_loss']]
    mag_loss = [metrics['simple_lstm_metrics']['magnitude_loss'], 
                metrics['attention_lstm_metrics']['magnitude_loss']]
    freq_loss = [metrics['simple_lstm_metrics']['frequency_loss'], 
                 metrics['attention_lstm_metrics']['frequency_loss']]
    
    ax2.bar(x - width/2, mag_loss, width, label='Magnitude Loss', alpha=0.8, color='lightblue')
    ax2.bar(x + width/2, freq_loss, width, label='Frequency Loss', alpha=0.8, color='salmon')
    ax2.set_xlabel('Model')
    ax2.set_ylabel('Loss')
    ax2.set_title('Hybrid Config: Loss Comparison', fontweight='bold')
    ax2.set_xticks(x)
    ax2.set_xticklabels(models)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Correlation Comparison
    mag_corr = [metrics['simple_lstm_metrics']['magnitude_corr'], 
                metrics['attention_lstm_metrics']['magnitude_corr']]
    freq_corr = [metrics['simple_lstm_metrics']['frequency_corr'], 
                 metrics['attention_lstm_metrics']['frequency_corr']]
    
    ax3.bar(x - width/2, mag_corr, width, label='Magnitude Correlation', alpha=0.8, color='lightgreen')
    ax3.bar(x + width/2, freq_corr, width, label='Frequency Correlation', alpha=0.8, color='orange')
    ax3.set_xlabel('Model')
    ax3.set_ylabel('Correlation')
    ax3.set_title('Hybrid Config: Correlation Comparison', fontweight='bold')
    ax3.set_xticks(x)
    ax3.set_xticklabels(models)
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Overall Performance Heatmap
    performance_data = [
        [mag_acc[0], freq_acc[0]],  # Simple LSTM
        [mag_acc[1], freq_acc[1]]   # Attention LSTM
    ]
    
    im = ax4.imshow(performance_data, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)
    ax4.set_xticks([0, 1])
    ax4.set_xticklabels(['Magnitude', 'Frequency'])
    ax4.set_yticks([0, 1])
    ax4.set_yticklabels(['Simple LSTM', 'Attention LSTM'])
    ax4.set_title('Hybrid Config: Performance Heatmap', fontweight='bold')
    
    # Add text annotations
    for i in range(2):
        for j in range(2):
            text = ax4.text(j, i, f'{performance_data[i][j]:.3f}',
                           ha="center", va="center", color="black", fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("📊 Hybrid configuration visualizations created successfully!")
    print("\n�� Key Insights:")
    print(f"  • Magnitude Performance: {'✅ Good' if min(mag_acc) > 0.6 else '⚠️ Needs improvement'}")
    print(f"  • Frequency Performance: {'✅ Good' if min(freq_acc) > 0.7 else '⚠️ Needs improvement'}")
    print(f"  • Overfitting Prevention: {'✅ Effective' if max(freq_acc) < 0.95 else '⚠️ May be overfitting'}")
    print(f"  • Model Balance: {'✅ Balanced' if abs(sum(mag_acc) - sum(freq_acc)) < 0.2 else '⚠️ Imbalanced'}")

## Save Hybrid Results to CSV

In [ ]:
if results and results.get('metrics'):
    metrics = results['metrics']
    
    # Create comprehensive summary DataFrame
    summary_df = pd.DataFrame({
        'Metric': ['Total Loss', 'Magnitude Loss', 'Frequency Loss', 'Magnitude Accuracy', 
                   'Frequency Accuracy', 'Magnitude Correlation', 'Frequency Correlation'],
        'Simple_LSTM': [
            metrics['simple_lstm_metrics']['total_loss'],
            metrics['simple_lstm_metrics']['magnitude_loss'],
            metrics['simple_lstm_metrics']['frequency_loss'],
            metrics['simple_lstm_metrics']['magnitude_accuracy'],
            metrics['simple_lstm_metrics']['frequency_accuracy'],
            metrics['simple_lstm_metrics']['magnitude_corr'],
            metrics['simple_lstm_metrics']['frequency_corr']
        ],
        'Attention_LSTM': [
            metrics['attention_lstm_metrics']['total_loss'],
            metrics['attention_lstm_metrics']['magnitude_loss'],
            metrics['attention_lstm_metrics']['frequency_loss'],
            metrics['attention_lstm_metrics']['magnitude_accuracy'],
            metrics['attention_lstm_metrics']['frequency_accuracy'],
            metrics['attention_lstm_metrics']['magnitude_corr'],
            metrics['attention_lstm_metrics']['frequency_corr']
        ]
    })
    
    # Save to CSV
    output_file = "hybrid_balanced_comparison_results.csv"
    summary_df.to_csv(output_file, index=False)
    
    print(f"✅ Results saved to: {output_file}")
    print("\n�� Results Summary:")
    print(summary_df.round(4))
    
    # Calculate differences and analysis
    summary_df['Difference'] = summary_df['Attention_LSTM'] - summary_df['Simple_LSTM']
    summary_df['Simple_LSTM_Wins'] = summary_df['Difference'] < 0
    
    print("\n📊 Performance Differences (Attention - Simple):")
    print(summary_df[['Metric', 'Difference']].round(4))
    
    # Count wins
    simple_wins = summary_df['Simple_LSTM_Wins'].sum()
    attention_wins = len(summary_df) - simple_wins
    
    print(f"\n🏆 Final Score:")
    print(f"  Simple LSTM wins: {simple_wins} metrics")
    print(f"  Attention LSTM wins: {attention_wins} metrics")
    
    if simple_wins > attention_wins:
        print(f"  🎯 OVERALL WINNER: Simple LSTM")
    elif attention_wins > simple_wins:
        print(f"  🎯 OVERALL WINNER: Attention LSTM")
    else:
        print(f"  🎯 TIE: Both models perform equally well")
    
    # Hybrid config assessment
    print(f"\n🔬 HYBRID CONFIG ASSESSMENT:")
    
    # Check if hybrid config achieved its goals
    mag_performance = min(metrics['simple_lstm_metrics']['magnitude_accuracy'], 
                          metrics['attention_lstm_metrics']['magnitude_accuracy'])
    freq_performance = min(metrics['simple_lstm_metrics']['frequency_accuracy'], 
                           metrics['attention_lstm_metrics']['frequency_accuracy'])
    max_freq = max(metrics['simple_lstm_metrics']['frequency_accuracy'], 
                   metrics['attention_lstm_metrics']['frequency_accuracy'])
    
    print(f"  Magnitude Target (70-85%): {'✅ Achieved' if 0.7 <= mag_performance <= 0.85 else '⚠️ Outside target'}")
    print(f"  Frequency Target (75-90%): {'✅ Achieved' if 0.75 <= freq_performance <= 0.9 else '⚠️ Outside target'}")
    print(f"  Overfitting Prevention: {'✅ Effective' if max_freq < 0.95 else '⚠️ May be overfitting'}")
else:
    print("⚠️  No metrics available to save.")

## Summary and Recommendations

In [ ]:
if results and results.get('metrics'):
    metrics = results['metrics']
    
    print("�� HYBRID CONFIG COMPREHENSIVE SUMMARY")
    print("=" * 70)
    print(f"Configuration: Hybrid Balanced Config")
    print(f"Analysis Type: Anti-Overfitting + High Performance Hybrid")
    print()
    
    # Overall performance
    print("🏆 OVERALL PERFORMANCE:")
    simple_mag_acc = metrics['simple_lstm_metrics']['magnitude_accuracy']
    simple_freq_acc = metrics['simple_lstm_metrics']['frequency_accuracy']
    attention_mag_acc = metrics['attention_lstm_metrics']['magnitude_accuracy']
    attention_freq_acc = metrics['attention_lstm_metrics']['frequency_accuracy']
    
    print(f"  Simple LSTM:     {simple_mag_acc:.3f} magnitude, {simple_freq_acc:.3f} frequency")
    print(f"  Attention LSTM:  {attention_mag_acc:.3f} magnitude, {attention_freq_acc:.3f} frequency")
    print()
    
    # Winner determination
    simple_total = simple_mag_acc + simple_freq_acc
    attention_total = attention_mag_acc + attention_freq_acc
    
    if attention_total > simple_total:
        print("🎯 WINNER: Attention LSTM")
        print(f"   Reason: Higher combined accuracy ({attention_total:.3f} vs {simple_total:.3f})")
    else:
        print("🎯 WINNER: Simple LSTM")
        print(f"   Reason: Higher combined accuracy ({simple_total:.3f} vs {attention_total:.3f})")
    
    print()
    
    # Hybrid config effectiveness
    print("🔬 HYBRID CONFIG EFFECTIVENESS:")
    max_acc = max(simple_mag_acc, simple_freq_acc, attention_mag_acc, attention_freq_acc)
    
    # Check anti-overfitting success
    if max_acc < 0.95:
        print(f"  ✅ Anti-Overfitting: Effective (max accuracy {max_acc:.3f} < 0.95)")
    else:
        print(f"  ⚠️ Anti-Overfitting: May need adjustment (max accuracy {max_acc:.3f} >= 0.95)")
    
    # Check performance balance
    mag_range = max(simple_mag_acc, attention_mag_acc) - min(simple_mag_acc, attention_mag_acc)
    freq_range = max(simple_freq_acc, attention_freq_acc) - min(simple_freq_acc, attention_freq_acc)
    
    print(f"  Magnitude consistency: {'✅ Good' if mag_range < 0.2 else '⚠️ High variance'}")
    print(f"  Frequency consistency: {'✅ Good' if freq_range < 0.2 else '⚠️ High variance'}")
    
    print()
    
    # Recommendations
    print("�� RECOMMENDATIONS:")
    
    # Performance recommendations
    if attention_total > simple_total + 0.1:
        print("  • Attention LSTM shows significant improvement - consider for production")
    elif simple_total > attention_total + 0.1:
        print("  • Simple LSTM performs better - simpler model may be sufficient")
    else:
        print("  • Both models perform similarly - choose based on computational requirements")
    
    # Config adjustment recommendations
    if max_acc < 0.8:
        print("  • Hybrid config is working well - realistic performance achieved")
    elif max_acc < 0.9:
        print("  • Hybrid config is balanced - good performance without overfitting")
    else:
        print("  • Consider reducing model capacity or increasing regularization")
    
    # Next steps
    print(f"\n🚀 NEXT STEPS:")
    print(f"  1. Run per-bin analysis to get spatial performance details")
    print(f"  2. Test on validation set to confirm generalization")
    print(f"  3. Deploy winning model to production")
    
    print()
    print("🎯 Hybrid configuration analysis completed successfully!")
    print("\nNote: This analysis shows overall model performance.")
    print("For per-bin accuracy analysis, you would need to run the trained models")
    print("on test data with spatial binning to get detailed per-bin metrics.")